In [2]:
from sentence_transformers import SentenceTransformer, util
import torch

m = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

In [25]:
def x_in_y(query, base):
    try:
        l = len(query)
    except TypeError:
        l = 1
        query = type(base)((query,))

    for i in range(len(base)):
        if base[i:i+l] == query:
            return list(range(i, i+l))
    return False

def sbert_sim(sents, phrase1, phrase2):

    if not (phrase1 and phrase2):
        return 0.0
    
    sents_ids = m.tokenize(sents)
    sents_embs = m.encode(sents_ids, convert_to_tensor=True,
                          is_pretokenized=True,
                          output_value='token_embeddings')
    
    tokenized1 = m.tokenize(phrase1)
    tokenized2 = m.tokenize(phrase2)
    
    print(sents_ids[0], len(sents_ids[0]))
    print(sents_ids[1], len(sents_ids[1])) 
    print(m.get_sentence_features(sents_ids[0], 12)['input_ids'])
    print(m.get_sentence_features(sents_ids[1], 12)['input_ids'])
    #print(sents_embs.shape)
    
    print(tokenized1, tokenized2)
    phrase1_embs_indices = x_in_y(tokenized1, sents_ids[0])
    phrase2_embs_indices = x_in_y(tokenized2, sents_ids[1])
    print(phrase1_embs_indices, phrase2_embs_indices)
    
    if not (phrase1_embs_indices and phrase2_embs_indices):
        return 0.0
    
    sents_embs_new = sents_embs[:, 1:].detach().clone() # Because embeddings include 'CLS'-token, but .tokenize() does not!
    
    return torch.nn.CosineSimilarity(0)(torch.mean(sents_embs_new[0, phrase1_embs_indices], 0), 
                                        torch.mean(sents_embs_new[1, phrase2_embs_indices], 0)).item()

#How do I pump up water pressure in my shower?	How can I boost the water pressure in my shower?

sents = ['How do I pump up water pressure in my shower?', 'How can I boost the water pressure in my shower?']

phrase1 = 'pump up'
phrase2 = 'boost'

print(sbert_sim(sents, phrase1, phrase2))

[2129, 2079, 1045, 10216, 2039, 2300, 3778, 1999, 2026, 6457, 1029] 11
[2129, 2064, 1045, 12992, 1996, 2300, 3778, 1999, 2026, 6457, 1029] 11
tensor([[  101,  2129,  2079,  1045, 10216,  2039,  2300,  3778,  1999,  2026,
          6457,  1029,   102,     0,     0]])
tensor([[  101,  2129,  2064,  1045, 12992,  1996,  2300,  3778,  1999,  2026,
          6457,  1029,   102,     0,     0]])
[10216, 2039] [12992]
[3, 4] [3]
0.813988208770752


In [9]:
print(ids, ids_query)
print(len(ids[0]))
print(x_in_y(ids_query, ids[0]))
print(embeddings1.shape)
#print(m.max_seq_length)

[[1996, 2126, 2008, 1037, 1000, 2744, 1000, 2003, 4225, 9041, 2006, 1996, 4646, 1012], [2057, 4521, 2413, 22201, 1012]] [1996, 4646]
14
[11, 12]
torch.Size([2, 17, 768])


In [6]:
a = ['apple', 'transformer']
apple, transformer = m.encode(a, convert_to_tensor=True)
apple.shape

torch.Size([768])

In [9]:
ids = m.tokenize([sents[0]])[0]
for i in ids:
    print(i, '->', m.tokenizer.decode([i])) 
    
print(m.tokenizer.decode([0]))
print(m.get_sentence_features(m.tokenize([sents[0]])[0], m.max_seq_length))

[1996, 4646]
1996 -> the
2126 -> way
2008 -> that
1037 -> a
1000 -> "
2744 -> term
1000 -> "
2003 -> is
4225 -> defined
9041 -> depends
2006 -> on
1996 -> the
4646 -> application
1012 -> .
[PAD]
{'input_ids': tensor([[ 101, 1996, 2126, 2008, 1037, 1000, 2744, 1000, 2003, 4225, 9041, 2006,
         1996, 4646, 1012,  102,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,

In [51]:
amr_file1 = 'analysis/sts/s2match_sbert/STS2016_corpus_a_reif.amr'
amr_file2 = 'analysis/sts/s2match_sbert/STS2016_corpus_b_reif.amr'

sbert_model = m

with open(amr_file1) as f1, open(amr_file2) as f2:
    amrs1 = f1.read().strip().split('\n\n')
    amrs2 = f2.read().strip().split('\n\n')

amr_len = min(len(amrs1), len(amrs2))
#amr_len = 16
amrs_sents1 = [amr.split('\n')[1].strip()[len('# ::snt '):] for amr in amrs1[:amr_len]]
amrs_sents2 = [amr.split('\n')[1].strip()[len('# ::snt '):] for amr in amrs2[:amr_len]]    

amrs_sents = [None]*2*amr_len
amrs_sents[::2] = amrs_sents1
amrs_sents[1::2] = amrs_sents2

sents_encoded = sbert_model.tokenize(amrs_sents)

sents_embs = sbert_model.encode(amrs_sents, convert_to_tensor=False,
                                convert_to_numpy=False,
                                output_value='token_embeddings',
                                is_pretokenized=False,
                                show_progress_bar=False)
#print(sents_embs)

In [53]:
s = []
for sent in sents_embs:
    s.append(sent.shape[0]) 

In [57]:
s = torch.nn.utils.rnn.pad_sequence(sents_embs, batch_first=True)
s.shape

torch.Size([1866, 39, 768])